# Sales Bot with OpenAI - A Summarization and RAG Use Case

## Overview

In this notebook you'll take an Amazon product reviews dataset from Kaggle and use OpenAI gpt-4o to obtain product review summaries, upsert those summaries in a vector database, then use Retrieval Augmented Generation (RAG) to power a sales chatbot that can make targeted product recommendations.

Let's take a look at the overall workflow:
1. We start with a dataset that contains over 10,000 reviews across 900 Amazon musical instruments and accessories.
2. Using gpt-4o chat, we generate summaries of product reviews for each product from the 20 most recent reviews. We format the summaries in JSON format.
3. We then take the summaries and upsert them into a vector database (Couchbase in this case)
4. We then use Couchbase vector Indexes and gpt-4o instruct to build a RAG-based sales chatbot that provides targeted recommendations to the user based on the products that are present in the inventory.

### OpenAI
We'll use [OpenAI](https://platform.openai.com/) to power all of the GenAI model needs of this notebook: LLMs, image gen, image animation.
* To use OpenAI model, you'll need to go to https://platform.openai.com/ and sign in .
* Next you'll need to generate an API token by following these [instructions](https://platform.openai.com/docs/quickstart). Keep the API token in hand, we'll need it further down in this notebook.

In this example we will use the gpt-4o instruct model.


### Couchbase
We'll use Couchbase Capella for our vector database. 


### Local Python Notebook

Open your terminal or command prompt and use the `cd` command to navigate to the directory where your Jupyter notebook is located. For example:

```bash
cd /path/to/your/notebook/directory
```

Use the `venv` module (or `virtualenv` if you prefer) to create a new virtual environment within that directory:

```bash
python -m venv .venv  # Creates a virtual environment named '.venv'
```

Activate the environment to start using it:

```bash
source .venv/bin/activate  # On Linux/macOS
.venv\Scripts\activate  # On Windows
```

This allows Jupyter to recognize your virtual environment:

```bash
pip install ipykernel
```

This makes your virtual environment selectable within Jupyter:

```bash
python -m ipykernel install --user --name=.venv --display-name="My Notebook Env"
```
(Replace "My Notebook Env" with a descriptive name for your kernel.)

* **Start Jupyter Notebook:**  `jupyter notebook`
* **Create a new notebook or open your existing one.**
* **Select the kernel:** Go to "Kernel" -> "Change kernel" and choose the kernel you just created ("My Notebook Env").

You don't need to install additional pip packages ahead of running the notebook, since those will be installed right at the beginning. You will need to ensure your system has `imagemagick` installed by following the [instructions](https://imagemagick.org/script/download.php).

In [1]:
# Let's start by installing the appropriate python packages
! pip install openai python-dotenv couchbase requests pandas gdown gradio pydantic langchain

  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached gradio-5.7.1-py3-none-any.whl.metadata (16 kB)
  Using cached pydantic-2.10.2-py3-none-any.whl.metadata (170 kB)
  Using cached langchain-0.3.9-py3-none-any.whl.metadata (7.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.4.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio

## Part 1: Review Summarization

Let's start by importing all of the packages we need for this example

In [2]:
import gradio
import json
import langchain
import os
import openai
import couchbase
import numpy
from getpass import getpass
from json import loads
from pandas import DataFrame, concat, read_csv
from pydantic import BaseModel, Field
from typing import List

Enter your OpenAI tokens below and the Couchbase host and credentials

In [3]:
OPENAI_API_KEY = "<your API key>"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

#openai
# 

In [4]:
# Get Couchbase User and Password
user = "application"
os.environ["COUCHBASE_USER"] = user
password = "Pwd12345!"
os.environ["COUCHBASE_PASSWORD"] = password

In [5]:
# Get Couchbase URL
couchbase_url = "couchbases://cb.ozutfofjnokedkp9.cloud.couchbase.com"
os.environ["COUCHBASE_URL"] = couchbase_url

In [6]:
# First let's load the dataset from Kaggle: https://www.kaggle.com/datasets/eswarchandt/amazon-music-reviews
df = read_csv('Musical_instruments_reviews.csv')

Set `product_record_limit` to a lower number if you just want to do a test run

In [7]:
# Set a product record limit
product_record_limit = 500

In [8]:
# List all of the unique ASIN:
asin_list = df.asin.unique()
print("There are {} unique products in the music product inventory".format(len(asin_list)))

There are 900 unique products in the music product inventory


For each one of the unique products, let's group the reviews together and sort them by how recent they are

In [9]:
# Get the reviews for the product ASIN, sorted by recency and store in dict
review_dict = {}
for asin in asin_list[0:product_record_limit]:
    reviews = df.loc[df['asin'] == asin]\
                .sort_values(["unixReviewTime"], axis=0, ascending=False)\
                .reviewText.tolist()
    review_dict[asin] = reviews

To be able to store our summaries into our vector DB, we need to have the fields formatted into a JSON object. We use Pydantic base class model here to define our formatting.

In [10]:
# Define the Pydantic model that specifies how our output should be formatted
class ProductRecord(BaseModel):
    """The record of a given product"""
    description: str = Field(description="Description of the product")
    name: str = Field(description="Name of the product")
    review_summary: str = Field(description="Summary of all of the reviews")
    ASIN: str = Field(description="ASIN of the product")
    features: str = Field(description="Features of the product based on the reviews")

We define our prompt template below.

In [11]:
# Prepare a prompt template
template = '''
Here are product reviews for a music product with an ID of {asin}.
 - Respond back only as only JSON!
 - Provide:
     - the product "description",
     - the product "name",
     - a summary of all the reviews as "review_summary",
     - the "ASIN" and
     - and the product "features" based on the content of these reviews.
 - The "features" should be a string describing the features and NOT JSON.
 - Do not include the ASIN in the description field.

The reviews for the product are: {reviews}
'''

We initialize the client using OpenAI APIs. All we have to do is override the `base_url` and `api_key`.

In [12]:
# Init OpenAI client
client = openai.OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"]
)

Iterate over all product ASINs and summarize the top 20 most recent reviews. Note: this takes a while to run unless we parallelize it.

In [12]:
# Produce the 900 product summaries
review_summaries = []
counter = 0

# This can take a while to process serially (30min+)
# TODO: Optimize to run in a few parallel threads to run faster while meeting the 240RPM limit
for asin, review_list in review_dict.items():
    print(f'Getting review summary {counter} of {len(review_dict)}, ASIN: {asin}')
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": template.format(
                    asin = asin,
                    reviews = review_list[0:20]
                )},
            ],
            temperature=0,
            max_tokens=1024,
            response_format={"type": "json_schema", "json_schema" : {"name": "hello", "schema": ProductRecord.model_json_schema()}}
            #
        )
        #print("\n{}\n".format(response.choices[0].message.content))
        summary = loads(response.choices[0].message.content)
        summary["ASIN"] = asin
        review_summaries.append(summary)
    except Exception as e:
        traceback.print_exc()
    counter += 1

review_summaries = DataFrame(review_summaries)

print(review_summaries.head())

Getting review summary 0 of 500, ASIN: 1384719342
Getting review summary 1 of 500, ASIN: B00004Y2UT
Getting review summary 2 of 500, ASIN: B00005ML71
Getting review summary 3 of 500, ASIN: B000068NSX
Getting review summary 4 of 500, ASIN: B000068NTU
Getting review summary 5 of 500, ASIN: B000068NVI
Getting review summary 6 of 500, ASIN: B000068NW5
Getting review summary 7 of 500, ASIN: B000068NZC
Getting review summary 8 of 500, ASIN: B000068NZG
Getting review summary 9 of 500, ASIN: B000068O1N
Getting review summary 10 of 500, ASIN: B000068O3D
Getting review summary 11 of 500, ASIN: B000068O3X
Getting review summary 12 of 500, ASIN: B000068O4H
Getting review summary 13 of 500, ASIN: B000068O59
Getting review summary 14 of 500, ASIN: B00006LVEU
Getting review summary 15 of 500, ASIN: B00009W40D
Getting review summary 16 of 500, ASIN: B00009W40G
Getting review summary 17 of 500, ASIN: B0000AQRSR
Getting review summary 18 of 500, ASIN: B0000AQRSS
Getting review summary 19 of 500, ASIN: B

# Part 2: Retrieval Augmented Generation

For our RAG use case we're going to rely on Couchbase vector database and on an OpenAI embedding model.

In [ ]:
from datetime import timedelta
import traceback
import json
# For exceptions
import couchbase
from couchbase.exceptions import CouchbaseException
# Required for any cluster connection
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
# Required for options -- cluster, timeout, SQL++ (N1QL) query, etc.
from couchbase.options import ClusterOptions, QueryOptions
from couchbase.management.collections import CollectionSpec
from couchbase.management.buckets import CreateBucketSettings
# Connect options - authentication
auth = PasswordAuthenticator(user, password)
# Get a reference to our cluster
options = ClusterOptions(auth)
# Use the pre-configured profile below to avoid latency issues with your connection.
options.apply_profile("wan_development")
try:
	cluster = Cluster(couchbase_url, options)
	# Wait until the cluster is ready for use.
	cluster.wait_until_ready(timedelta(seconds=5))
except Exception as e:
	traceback.print_exc()


bucket = cluster.bucket("hello")
scope = bucket.scope("_default")
collection = scope.collection("_default")


Now we upsert all of the vectors into the databse using OpenAI embedding model.

In [ ]:
# Update embeddings to use OpenAI embeddings
import requests
import json
import os 

OPENAI_API_KEY = "<your API key>"
headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
}

def get_embedding(text, model="text-embedding-3-small",token=OPENAI_API_KEY):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    text = text.replace("\n", " ")
    embed_url = "https://api.openai.com/v1/embeddings"
    data = {
        "input": text,
        "model": model
    }
    response = requests.post(embed_url, headers=headers, data=json.dumps(data))
    res = response.json()
    return res["data"][0]['embedding']

In [ ]:
 review_summaries.head(50)

In [16]:
for index, row in review_summaries.iterrows():
        doc_id = f"{index}"
        upsert_dict = row.to_dict()
       # print(row['REVIEWTEXT'])

        try:
            # Vectorize SUMMARY and REVIEWTEXT
            upsert_dict["description_embedding"] = get_embedding(row["description"][0:9999])
            upsert_dict["review_summary_embedding"] = get_embedding(row["review_summary"][0:9999])
            upsert_dict["features_embedding"] = get_embedding(row["features"][0:9999])
            #print(upsert_dict)
            collection.upsert(doc_id, upsert_dict)
            print(f"Upserted {doc_id}")

        except Exception as e:
            print(f"Error upserting {doc_id}")
            traceback.print_exc()

Upserted 0
Upserted 1
Upserted 2
Upserted 3
Upserted 4
Upserted 5
Upserted 6
Upserted 7
Upserted 8
Upserted 9
Upserted 10
Upserted 11
Upserted 12
Upserted 13
Upserted 14
Upserted 15
Upserted 16
Upserted 17
Upserted 18
Upserted 19
Upserted 20
Upserted 21
Upserted 22
Upserted 23
Upserted 24
Upserted 25
Upserted 26
Upserted 27
Upserted 28
Upserted 29
Upserted 30
Upserted 31
Upserted 32
Upserted 33
Upserted 34
Upserted 35
Upserted 36
Upserted 37
Upserted 38
Upserted 39
Upserted 40
Upserted 41
Upserted 42
Upserted 43
Upserted 44
Upserted 45
Upserted 46
Upserted 47
Upserted 48
Upserted 49
Upserted 50
Upserted 51
Upserted 52
Upserted 53
Upserted 54
Upserted 55
Upserted 56
Upserted 57
Upserted 58
Upserted 59
Upserted 60
Upserted 61
Upserted 62
Upserted 63
Upserted 64
Upserted 65
Upserted 66
Upserted 67
Upserted 68
Upserted 69
Upserted 70
Upserted 71
Upserted 72
Upserted 73
Upserted 74
Upserted 75
Upserted 76
Upserted 77
Upserted 78
Upserted 79
Upserted 80
Upserted 81
Upserted 82
Upserted 83
Up

In [ ]:
# Inspect our processed review data with the embeddings for SUMMARY and REVIEWTEXT
import pandas as pd
result = cluster.query("SELECT * FROM `hello`.`_default`.`_default` limit 5")
pd.json_normalize(pd.DataFrame(result)['_default']).head()

# Configure a (large) Vector Search Index on our CB Collection
Before our Vectors are available to semantic search, we have to configure a Vector Search Index on our new embeddings columns. The simplest way to do this is via Couchbase's Search Web Console. 

## Using the Search Web Console
1. Navigate Data Tools / Search
2. Create a name for your new index, such as, `products_emb_idx`.
3. Choose your Bucket, Scope, and collection from the search filter (e.g. `hello._default._default`.)
4. From columns list, select your embeddings column, `review_summary_embedding`. In the "Configure New Type Mapping" pane to the left, you will see mappings pre-filled for the selected embeddings column. You do not need to alter these. Click "Add". You can add the three vector embeddings we created before and any other field as an fts field. Then mark the fts fields to be stored in the result.
5. Follow the same steps to add additional fields to your search. You may wish to include other information in your semantic search, including the raw text review_summary and description strings. 
6. Click "Create Index" to create the next Index. 

[Couchbase Official Docs - Create a Vector Search Index with Server Web Console](https://docs.couchbase.com/server/current/vector-search/create-vector-search-index-ui.html)

# Run a semantic search query on the processed reviews
We can now run a semantic search query on our processed reviews. This is an essential step to generating accurate and tailored results using large language models. Let's run a search positive reviews of guitars. 

In [ ]:
import couchbase.search as search
from couchbase.options import SearchOptions
from couchbase.vector_search import VectorQuery, VectorSearch

# https://docs.couchbase.com/server/current/vector-search/run-vector-search-sdk.html

SEARCH_QUERY = "nice guitars"
search_index = "products_emb_idx"
num_results = 5
try:
    vector = get_embedding(SEARCH_QUERY) # Create an embedding of the SEARCH_QUERY
    search_req = search.SearchRequest.create(search.MatchNoneQuery()).with_vector_search(
        VectorSearch.from_vector_query(VectorQuery('review_summary_embedding', vector, num_candidates=num_results)))
        # Change the limit value to return more results. Change the fields array to return different fields from your Search index.
    result = scope.search(search_index, search_req, SearchOptions(limit=num_results, fields=["review_summary","description","ASIN", "name"]))

    for row in result.rows():
        print(json.dumps(row.fields))
        
    
except CouchbaseException as ex:
    import traceback
    traceback.print_exc()

Let's now try to run a hybrid search on the following query below.
Hybrid search combines the results of a vector search and a keyword (BM25F) search by fusing the two result sets.
It will return the 3 closest entries in the database according to the search criteria.

In [20]:
# Hybrid search
SEARCH_QUERY = "best karaoke mic"
search_index = "products_emb_idx"
num_results = 3
try:
    vector = get_embedding(SEARCH_QUERY) # Create an embedding of the SEARCH_QUERY
    search_req = search.SearchRequest.create(search.MatchPhraseQuery(SEARCH_QUERY)).with_vector_search(
        VectorSearch([VectorQuery.create('review_summary_embedding', vector, num_candidates=num_results),
                          VectorQuery.create('description_embedding', vector, num_candidates=num_results),
                          VectorQuery.create('features_embedding', vector, num_candidates=num_results)]))
    
    # Change the limit value to return more results. Change the fields array to return different fields from your Search index.
    result = scope.search(search_index, search_req, SearchOptions(limit=num_results, fields = ["review_summary","description","ASIN", "name", "features"]))
    
    for row in result.rows():
        print(json.dumps(row.fields))
        
except CouchbaseException as ex:
    import traceback
    traceback.print_exc()

{"ASIN": "B000N5YEDG", "description": "The Nady DKW Duo is a dual channel VHF handheld microphone system designed for karaoke, DJ, and live sound applications. It offers good audio quality and affordability, making it a popular choice for budget-conscious users.", "features": "Affordable dual channel VHF wireless microphone system, good audio quality, easy setup, changeable mesh balls, requires direct line of sight for best performance, limited upper frequency response, long range, suitable for karaoke and DJ use.", "name": "Nady DKW Duo Dual Channel VHF Handheld Microphone System"}
{"ASIN": "B000WS1QC6", "description": "The Yamaha PA130 AC Power Adapter is designed to power Yamaha keyboards, providing a reliable and lightweight alternative to using batteries.", "features": "Lightweight and compact design, suitable for Yamaha keyboards, provides reliable power, prevents the need for batteries, fits perfectly and stays secure, more affordable than store options, fast shipping.", "name":

Let's now define a helper function that gives us the relevant context given a string query. Let's see what it returns based on the question: "What is a good beginner harmonica"

In [21]:
def get_context(question, limit=3):
    num_results = 3
    try:
        vector = get_embedding(question) # Create an embedding of the SEARCH_QUERY
        search_req = search.SearchRequest.create(search.MatchPhraseQuery(question)).with_vector_search(
            VectorSearch([VectorQuery.create('review_summary_embedding', vector, num_candidates=num_results),
                          VectorQuery.create('description_embedding', vector, num_candidates=num_results),
                          VectorQuery.create('features_embedding', vector, num_candidates=num_results)]))
            # Change the limit value to return more results. Change the fields array to return different fields from your Search index.
        result = scope.search(search_index, search_req, SearchOptions(limit=num_results,fields=["review_summary","description","ASIN", "name", "features"]))
        
        return [row.fields for row in result.rows()]
    except CouchbaseException as ex:
        import traceback
        traceback.print_exc()


In [22]:
print(json.dumps(get_context("What is a good beginner harmonica")))

[{"ASIN": "B000VUNEZM", "description": "A padded guitar strap designed for comfort and durability, suitable for electric and acoustic guitars.", "features": "Padded for comfort, suitable for electric and acoustic guitars, affordable, easy to install and remove, may require strap locks for horizontal pegs, padding helps the strap lay flat across the shoulder, good for entry-level or backup use.", "name": "Padded Guitar Strap"}, {"ASIN": "B0006GR4L6", "description": "A high-quality leather guitar strap with a weathered and aged look, designed for comfort and durability.", "features": "The guitar strap is made of high-quality leather with a weathered and aged texture. It is wide and comfortable, suitable for holding heavy guitars. The strap is durable and expected to last a long time. It is soft without requiring a break-in period, though some users find it a bit short for larger guitars. The strap is simple yet attractive, making it a great accessory for both electric and acoustic guitar

Great, we're now ready to build a sales assistant helper function.

We first define a prompt template for Llama 3 - based on the context provided by the vector hybrid search (i.e. collection of product summaries of relevance to the question), provide a helpful recommendation to the customer.

Also provide links to the product that the user can click on to view the product on Amazon's website. For that we use the fact that any product referenced by its aSIN can be accessed at the following url: `https://www.amazon.com/exec/obidos/ASIN/<insert aSIN here>`

In [23]:
sales_template = """
You are a sales assistant. Answer the user questions as helpfully as possible.
Only recommend the products that are provided in the context provided below.

Provide a reference to each product you mention with hyperlinks:
* Provide the name of the product
* Embed the hyperlink in the name of the product as follows
    * If the product name is "Solid Electric Guitar Case with Accessories Compartment"
    * And the aSIN is "B001EL6I8W"
    * Format the reference as follows:
         [Solid Electric Guitar Case with Accessories Compartment](https://www.amazon.com/exec/obidos/ASIN/B001EL6I8W)

Finish with a references section.

Customer question: {}

Product context: {}

AI:
"""

def sales_assistant(question):
    response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": sales_template.format(question, get_context(question, limit=10))},
                ],
                temperature=0,
                max_tokens=1024
            )

    return response.choices[0].message.content

print(sales_assistant("what is must have accessory for my new piano?"))

A must-have accessory for your new piano is a sustain pedal, which enhances your playing by allowing notes to resonate longer, similar to an acoustic piano. I recommend the [M-Audio SP-2 Universal Sustain Pedal](https://www.amazon.com/exec/obidos/ASIN/B00063678K). It features a classic piano-style design, providing a natural and realistic pedal action. It's compatible with most keyboards due to its polarity switch and has a durable construction to withstand rigorous use.

Additionally, having a comfortable and adjustable bench can greatly improve your playing experience. You might consider the [On Stage KT7800 Keyboard Bench](https://www.amazon.com/exec/obidos/ASIN/B000VJ2VCK), which offers a padded seat and adjustable height settings, or the [On-Stage KB8902B Keyboard Bench](https://www.amazon.com/exec/obidos/ASIN/B000GUR8V8), known for its thick padding and sturdy construction.

### References
- [M-Audio SP-2 Universal Sustain Pedal](https://www.amazon.com/exec/obidos/ASIN/B00063678K

In this section we build a simple an interactive sales bot assistant using Gradio.

In [25]:
import gradio as gr

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human})
        history_openai_format.append({"role": "assistant", "content": assistant})
    history_openai_format.append({"role": "user", "content": sales_template.format(message, get_context(message, limit=5))})

    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = history_openai_format,
        temperature = 0.0,
        stream = True
     )

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message

gr.ChatInterface(predict).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
